In [ ]:
import numpy as np
import pandas as pd

import torch
import torchaudio

import librosa

if torchaudio.get_audio_backend() == 'soundfile':
    print("Warning: sox_io not available, using soundfile instead, this may cause some issue when loading .mp3")

import IPython.display as ipd

In [ ]:
from matplotlib import pyplot as plt
from librosa.display import waveshow, specshow

### FFT, STFT and MelSpectrogram

Convert Signal from Time Domain to Frequency Domain

![Time_to_Fre](https://upload.wikimedia.org/wikipedia/commons/6/61/FFT-Time-Frequency-View.png)

In [ ]:
def generate_pure_tone(freq, duration=1, sample_rate=44100, volumn=1):
    number_samples = duration * sample_rate
    samples = np.arange(0, number_samples)
    pr = 1 / sample_rate
    signal = volumn * np.sin(2 * np.pi * freq * samples * pr)
    return signal


In [ ]:
offset = 256 # Hz
sample_rate = 44100

f128 = generate_pure_tone(128, duration=1, sample_rate=sample_rate, volumn=1)
f128_offset = generate_pure_tone(128+offset, duration=1, sample_rate=sample_rate, volumn=1)

f1024 = generate_pure_tone(1024, duration=1, sample_rate=sample_rate, volumn=0.25)
f1024_offset = generate_pure_tone(1024+offset, duration=1, sample_rate=sample_rate, volumn=0.75)

# put all signals on top of each other
signal_stack = np.array([f128, f128_offset, f1024, f1024_offset])
signal_mono = f128 + f128_offset + f1024 + f1024_offset

# Put all signals on a sequence
signal_sequence = np.concatenate([f128, f128_offset, f1024, f1024_offset])
print(signal_sequence.shape)
print(signal_stack.shape)
print(signal_mono.shape)

In [ ]:
def plot_waveform(signal: np.ndarray, sample_rate:int, offset:int=0, n_frame:int=None, return_plot:bool=False):
    if len(signal.shape) == 1:
        signal = np.expand_dims(signal, axis=0)
    if n_frame is not None:
        signal = signal[:,offset:offset+n_frame]

    num_channels, num_frames = signal.shape
    
    fig, axes = plt.subplots(nrows=num_channels, ncols=1)

    if num_channels == 1:
        axes = [axes]
    for c in range(num_channels):
        waveshow(signal[c], sr=sample_rate, ax=axes[c])
        axes[c].grid(True)
        axes[c].label_outer()
        if num_channels > 1:
            axes[c].set_ylabel(f"Channel {c+1}")
    fig.suptitle("waveform")
    if return_plot:
        return fig, axes
    plt.show(block=False)


In [ ]:
# plot the first 1000 frames
plot_waveform(signal=signal_stack, sample_rate=sample_rate, offset=0, n_frame=1000)
plot_waveform(signal_mono, sample_rate, 0, 1000)

In [ ]:
display(ipd.Audio(data=signal_mono, rate=sample_rate))
display(ipd.Audio(data=signal_stack, rate=sample_rate))
display(ipd.Audio(data=signal_sequence, rate=sample_rate))

In [ ]:
def plot_magnitude_spectrum(signal: np.ndarray, sample_rate:int, offset:int=0, n_frame:int=None, f_ratio=1):
    if len(signal.shape) == 1:
        signal = np.expand_dims(signal, axis=0)
    if n_frame is not None:
        signal = signal[:,offset:offset+n_frame]
    
    # Apply FFT & Calculate the Magnitude Spectrum
    ft = np.fft.fft(signal)
    magnitude_spectrum = np.abs(ft)

    num_channels, num_frames = signal.shape
    figure, axes = plt.subplots(nrows=num_channels, ncols=1)

    frequency = np.linspace(0, sample_rate, num_frames)
    num_frequency_bins = int(len(frequency)*f_ratio)

    if num_channels == 1:
        axes = [axes]
    for c in range(num_channels):
        axes[c].plot(frequency[:num_frequency_bins], magnitude_spectrum[c,:num_frequency_bins])
        axes[c].set_xlabel('Frequency (Hz)')
        axes[c].grid(True)
        axes[c].label_outer()
        if num_channels > 1:
            axes[c].set_ylabel(f"Channel {c+1}")
    figure.suptitle("Magnitude Spectrum")
    plt.show(block=False)

In [ ]:
plot_magnitude_spectrum(signal_mono, sample_rate, f_ratio=0.1)

In [ ]:
stft_params = {
    'n_fft': 1024,
    'win_length': None,
    'hop_length': 512,
    'window': "hann",
    'center': True,
    'dtype': None,
    'pad_mode': "constant",
}

S = librosa.feature.melspectrogram(y=signal_sequence, sr=sample_rate, **stft_params, n_mels=108)
S_log = librosa.power_to_db(S, ref=np.max)

fig, ax = plt.subplots(1,1)
ax.set_title('mel spectrogram')
im = specshow(S_log, sr=sample_rate, x_axis='time', y_axis='mel', ax=ax)
fig.colorbar(im, ax=ax, format="%+2.0f dB")
plt.show()


# Audio Example

### Metadata

In [ ]:
audio_path = '../medleydb/medleydb/data/Audio/Phoenix_ScotchMorris/Phoenix_ScotchMorris_MIX.wav'
metadata = torchaudio.info(audio_path)
song_duration = metadata.num_frames/metadata.sample_rate # in seconds
m,s = divmod(song_duration, 60)
print(metadata)
print(f'Song duration: {song_duration:.2f}s ({m:.0f}m{s:.0f}s)')

### Load Audio

In [ ]:
# display Audio
ipd.Audio(filename=audio_path)

In [ ]:
# Load audio on memory using Librosa
# Note:
# - By default, librosa.load() resample audio to 22050Hz by default.
# - Output is a mono waveform.
librosa_waveform, sr = librosa.load(path=audio_path, sr=metadata.sample_rate)
print(librosa_waveform.shape)
print(sr)

In [ ]:
# Load audio on memory using pytorch
# Note:
# - Unlike librosa, pytorch use the sampling rate stored in the file.
# - Include multiple channels
# - Output a torch array, which can be converted into numpy.

# For Windows user: if torchaudio cannot load `.mp3` format, use librosa instead.
pytorch_waveform, sr = torchaudio.load(audio_path)
print(pytorch_waveform.shape)
print(sr)
#convert to mono waveform using mean():
waveform_mono = torch.mean(pytorch_waveform, dim=0)
print(f'1:1 comparison with Librosa waveform:', 'Matched' if np.all(waveform_mono.numpy() == librosa_waveform) else 'Unmatched')


### Data inspection

In [ ]:
plot_waveform(pytorch_waveform.numpy(),sr, 0, 5*sr)

In [ ]:
# Magnitude Spectrum of the whole song
plot_magnitude_spectrum(pytorch_waveform.numpy(), sr, f_ratio=1)
# Note:
# - The Frequency distribution is repeated and mirrored, this is called the redundancies
# - To remove the redundancies, simply set `f_ratio=0.5`

In [ ]:
# If we take the specstrum of the whole song, it doesn't give us much info
# Introducing Short time fourier transform
offset = 0
n_frame = 3*sr
signal = librosa_waveform[offset:offset+n_frame]

spec = librosa.stft(y=signal, **stft_params)
magnitude = librosa.amplitude_to_db(np.abs(spec), ref=np.max)
print(magnitude.shape)

# plot the spectrogram
fig, ax = plt.subplots(1,1)
ax.set_title('Power spectrogram')
im = specshow(magnitude, sr=sr, x_axis='time', y_axis='linear', ax=ax)
fig.colorbar(im, ax=ax, format="%+2.0f dB")
plt.show()


## Mel spectrogram

In [ ]:
mel_spec = librosa.feature.melspectrogram(y=signal, sr=sr, **stft_params, n_mels=108)
log_mel_spec = librosa.power_to_db(mel_spec, ref=np.max)
print(mel_spec.shape)
print(log_mel_spec.shape)
# plot the spectrogram
fig, ax = plt.subplots(1,1)
ax.set_title('mel spectrogram')
im = specshow(log_mel_spec, sr=sr, x_axis='time', y_axis='mel', ax=ax)
fig.colorbar(im, ax=ax, format="%+2.0f dB")
plt.show()